# Keras Functional  and Sequential APIの使い方

## Preparcing Tensorflow2.x and Dataset MNIST

In [6]:
import tensorflow as tf
import keras

# Mnist
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalization 0-255の値が入っているので、0-1に収まるよう正規化します
x_train, x_test = x_train / 255.0, x_test / 255.0

# Check the data
# Now, each row has one data
print(x_train.shape, x_test.shape)

(60000, 28, 28) (10000, 28, 28)


# Sequentialの場合の書き方

### ・keras.models.Sequential()にlistで与える

### ・model.add()で1層ずつ足してくかしてモデルをつくる

### 最期に学習条件を決めてcompileすれば完成です。


In [9]:
# Sequentialモデルを定義します

model = keras.models.Sequential()

model.add(tf.keras.layers.Flatten(input_shape =(28, 28)))
model.add(tf.keras.layers.Dense(128, activation ='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(10, activation ='softmax'))

# モデルをcompileします
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
display(model.summary())

# 学習します
hist = model.fit(x_train, y_train, validation_split=0.1, epochs=5)

# テストデータの予測精度を計算します
print(model.evaluate(x_test, y_test))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


None

Epoch 1/5
1688/1688 [==============================] - 2s 957us/step - loss: 0.3198 - accuracy: 0.9070 - val_loss: 0.1254 - val_accuracy: 0.9678
Epoch 2/5
1688/1688 [==============================] - 1s 818us/step - loss: 0.1527 - accuracy: 0.9547 - val_loss: 0.0944 - val_accuracy: 0.9730
Epoch 3/5
1688/1688 [==============================] - 1s 817us/step - loss: 0.1131 - accuracy: 0.9656 - val_loss: 0.0790 - val_accuracy: 0.9767
Epoch 4/5
1688/1688 [==============================] - 1s 840us/step - loss: 0.0914 - accuracy: 0.9724 - val_loss: 0.0713 - val_accuracy: 0.9792
Epoch 5/5
313/313 [==============================] - 0s 613us/step - loss: 0.0716 - accuracy: 0.9782
[0.07159699499607086, 0.9782000184059143]


# Functional の場合の書き方

## ①入力が１つの場合

### 上記のSequentialの場合とまったく同じモデルをfunctional APIで書くと次のようになります。

Sequentialだと入力数と出力数がどちらも１つと決まってるのでSequentialでネットワーク構造を定義したら完成でしたが、functional APIだと入力と出力をどちらも複数設定できますので、ネットワーク構造をkeras.layersで定義する部分の２つを書いておいて、入力と出力がいくつあるのかkeras.Model()で定義して完成となります。

In [11]:
# モデル構造を定義します
inputs = tf.keras.layers.Input(shape=(28, 28))
x = tf.keras.layers.Flatten()(inputs)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
predictions = tf.keras.layers.Dense(10, activation='softmax')(x)

# 入出力を定義します
model = keras.Model(inputs=inputs, outputs=predictions)

# モデルをcompileします
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
display(model.summary())

# 学習します
hist = model.fit(x_train, y_train, validation_split=0.1, epochs=5)

# テストデータの予測精度を計算します
print(model.evaluate(x_test, y_test))

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               100480    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


None

Epoch 1/5
1688/1688 [==============================] - 2s 903us/step - loss: 0.3117 - accuracy: 0.9086 - val_loss: 0.1193 - val_accuracy: 0.9677
Epoch 2/5
1688/1688 [==============================] - 1s 800us/step - loss: 0.1533 - accuracy: 0.9540 - val_loss: 0.0920 - val_accuracy: 0.9728
Epoch 3/5
1688/1688 [==============================] - 1s 799us/step - loss: 0.1156 - accuracy: 0.9654 - val_loss: 0.0822 - val_accuracy: 0.9760
Epoch 4/5
1688/1688 [==============================] - 1s 800us/step - loss: 0.0945 - accuracy: 0.9707 - val_loss: 0.0775 - val_accuracy: 0.9788
Epoch 5/5
313/313 [==============================] - 0s 519us/step - loss: 0.0782 - accuracy: 0.9768
[0.0782335177063942, 0.9768000245094299]


(Note)

学習データとテストデータのようにkerasの外からkerasモデルに渡すデータは必ず最初にkeras.layers.Input()で受け取り、そこから加える層の右にその層への入力を（）付きで与えるように書いて、1層ずつ増やしていくという書き方になります。

下の例だとpredictionsに入力から出力までのInput => Flatten => Dense(128, relu) => Dropout => Dense(10, softmax)までのネットワークが全部入ってますので、Sequentialで書いたmodelと同じ内容になります

## ② 入力が2つある場合(出力は１つ)

入力が複数ある場合はinputが複数あるネットワークを書いて、keras.Model()にlistでinputを与えるようにします。下の例はmnistデータを2つに分けてkeras model内で結合してから同じネットワークに通すようにしたものです。

In [12]:
# 複数入力のテストの為にxを分割してみます
# 全ての行、３９２
x_train2_1 = x_train.reshape(60000, 784)[:,:392]  # (60000, 392) 
x_train2_2 = x_train.reshape(60000, 784)[:,392:]  # (60000, 392)

x_test2_1 = x_test.reshape(10000, 784)[:,:392] # (10000, 392)
x_test2_2 = x_test.reshape(10000, 784)[:,392:] # (10000, 392)

# Functional APIでモデルを定義します
input1 = tf.keras.layers.Input(shape=(392,))
input2 = tf.keras.layers.Input(shape=(392,))
inputs = tf.keras.layers.concatenate([input1, input2])

x = tf.keras.layers.Dense(128, activation='relu')(inputs)
x = tf.keras.layers.Dropout(0.2)(x)
predictions = keras.layers.Dense(10, activation='softmax')(x)

# 入出力を定義します
model = tf.keras.Model(inputs=[input1, input2], outputs=predictions)

# モデルをcompileします
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
display(model.summary())

# 学習します
hist = model.fit([x_train2_1, x_train2_2], y_train, validation_split=0.1, epochs=5)

# テストデータの予測精度を計算します
print(model.evaluate([x_test2_1, x_test2_2], y_test))

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 392)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 392)]        0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 784)          0           input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
dense_8 (Dense)                 (None, 128)          100480      concatenate[0][0]     

None

Epoch 1/5
1688/1688 [==============================] - 2s 939us/step - loss: 0.3168 - accuracy: 0.9078 - val_loss: 0.1353 - val_accuracy: 0.9628
Epoch 2/5
1688/1688 [==============================] - 1s 847us/step - loss: 0.1555 - accuracy: 0.9542 - val_loss: 0.0946 - val_accuracy: 0.9733
Epoch 3/5
1688/1688 [==============================] - 1s 843us/step - loss: 0.1154 - accuracy: 0.9648 - val_loss: 0.0830 - val_accuracy: 0.9755
Epoch 4/5
1688/1688 [==============================] - 1s 868us/step - loss: 0.0947 - accuracy: 0.9707 - val_loss: 0.0758 - val_accuracy: 0.9782
Epoch 5/5
313/313 [==============================] - 0s 570us/step - loss: 0.0805 - accuracy: 0.9760
[0.08050625026226044, 0.9760000109672546]


## ③ 入力と出力が2つある場合（損失関数は１つ）

分岐を加えて出力が2つあるmodelに変えてみました。x1とx2の2つの経路に分岐していて、prediction1とprediction2がそれぞれの出力までのネットワーク情報をもっています。出力段が2つになったのでkeras.Model()に与える出力段も２つになります。

In [35]:
# 複数入力のテストの為にxを分割してみます
x_train2_1 = x_train.reshape(60000, 784)[:,:392]
x_train2_2 = x_train.reshape(60000, 784)[:,392:]
x_test2_1 = x_test.reshape(10000, 784)[:,:392]
x_test2_2 = x_test.reshape(10000, 784)[:,392:]

# Functional APIでモデルを定義します
input1 = keras.layers.Input(shape=(392,))
input2 = keras.layers.Input(shape=(392,))

# Prediction 1
inputs1 = keras.layers.concatenate([input1, input2])
x1 = keras.layers.Dense(128, activation='relu')(inputs1)
x1 = keras.layers.Dropout(0.2)(x1)
prediction1 = keras.layers.Dense(10, activation='softmax')(x1)

# Prediction 2
inputs2 = keras.layers.concatenate([input1, input2])
x2 = keras.layers.Dense(128, activation='relu')(inputs2)
x2 = keras.layers.Dropout(0.2)(x2)
prediction2 = keras.layers.Dense(10, activation='softmax')(x2)

# 入出力を定義します
model = keras.Model(inputs=[input1, input2], outputs=[prediction1, prediction2])

# モデルをcompileします
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
display(model.summary())

# 学習します
hist = model.fit([x_train2_1, x_train2_2], [y_train, y_train], 
                 validation_split=0.1, epochs=5)

# テストデータの予測精度を計算します
print(model.evaluate([x_test2_1, x_test2_2], [y_test, y_test]))

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 392)]        0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 392)]        0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 784)          0           input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 784)          0           input_5[0][0]         

None

Epoch 1/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.6246 - dense_11_loss: 0.3107 - dense_13_loss: 0.3139 - dense_11_accuracy: 0.9101 - dense_13_accuracy: 0.9089 - val_loss: 0.2544 - val_dense_11_loss: 0.1275 - val_dense_13_loss: 0.1268 - val_dense_11_accuracy: 0.9668 - val_dense_13_accuracy: 0.96673 - dense_11_loss: 0.4245 - dense_13_loss: 0.4308 - dense_11_accuracy: 0.8755 - dense
Epoch 2/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.3003 - dense_11_loss: 0.1488 - dense_13_loss: 0.1516 - dense_11_accuracy: 0.9551 - dense_13_accuracy: 0.9541 - val_loss: 0.1972 - val_dense_11_loss: 0.0994 - val_dense_13_loss: 0.0977 - val_dense_11_accuracy: 0.9712 - val_dense_13_accuracy: 0.9713
Epoch 3/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.2271 - dense_11_loss: 0.1128 - dense_13_loss: 0.1143 - dense_11_accuracy: 0.9655 - dense_13_accuracy: 0.9651 - val_loss: 0.1647 - val_dense_11_loss: 0.0829 - val_dense_13_loss: 0.0818 - va

# ④ 入力、出力、損失関数が2つある場合

せっかく出力を分けたので損失関数も別々に入れてみます。

modelを作るときにname=''で名付けておいて、compile()するときにlossを辞書型で渡せば出力ごとに異なる損失関数を使うことができます。下の例だと同じ損失関数を使ってますが、ぜんぜん違う損失関数を指定しても構いません。

学習はトータルの損失関数を最小化するように進めますがデフォルトでは単純に合計するようです。加算比率をloss_weightsに辞書型で渡すことで指定することもできるので、以下では0.5ずつで加算するようにしています。

In [36]:
# 複数入力のテストの為にxを分割してみます
x_train2_1 = x_train.reshape(60000, 784)[:,:392]
x_train2_2 = x_train.reshape(60000, 784)[:,392:]
x_test2_1 = x_test.reshape(10000, 784)[:,:392]
x_test2_2 = x_test.reshape(10000, 784)[:,392:]

# Functional APIでモデルを定義します
input1 = keras.layers.Input(shape=(392,))
input2 = keras.layers.Input(shape=(392,))

# Prediction 1
inputs1 = keras.layers.concatenate([input1, input2])
x1 = keras.layers.Dense(128, activation='relu')(inputs1)
x1 = keras.layers.Dropout(0.2)(x1)
prediction1 = keras.layers.Dense(10, activation='softmax', name='prediction1')(x1)

# Prediction 2
inputs2 = keras.layers.concatenate([input1, input2])
x2 = keras.layers.Dense(128, activation='relu')(inputs2)
x2 = keras.layers.Dropout(0.2)(x2)
prediction2 = keras.layers.Dense(10, activation='softmax', name='prediction2')(x2)

# 入出力を定義します
model = keras.Model(inputs=[input1, input2], outputs=[prediction1, prediction2])


# モデルをcompileします
model.compile(optimizer='adam',
              loss={'prediction1': 'sparse_categorical_crossentropy', 
                    'prediction2': 'sparse_categorical_crossentropy'},
              loss_weights={'prediction1': 0.5,
                            'prediction2': 0.5},
              metrics=['accuracy'])
display(model.summary())

# 学習します
hist = model.fit([x_train2_1, x_train2_2], [y_train, y_train], 
                 validation_split=0.1, epochs=5)

# テストデータの予測精度を計算します
print(model.evaluate([x_test2_1, x_test2_2], [y_test, y_test]))

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 392)]        0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 392)]        0                                            
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 784)          0           input_7[0][0]                    
                                                                 input_8[0][0]                    
__________________________________________________________________________________________________
concatenate_4 (Concatenate)     (None, 784)          0           input_7[0][0]         

None

Epoch 1/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.3114 - prediction1_loss: 0.3126 - prediction2_loss: 0.3103 - prediction1_accuracy: 0.9096 - prediction2_accuracy: 0.9100 - val_loss: 0.1248 - val_prediction1_loss: 0.1257 - val_prediction2_loss: 0.1239 - val_prediction1_accuracy: 0.9652 - val_prediction2_accuracy: 0.9643
Epoch 2/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.1493 - prediction1_loss: 0.1501 - prediction2_loss: 0.1485 - prediction1_accuracy: 0.9555 - prediction2_accuracy: 0.9565 - val_loss: 0.0983 - val_prediction1_loss: 0.0995 - val_prediction2_loss: 0.0971 - val_prediction1_accuracy: 0.9702 - val_prediction2_accuracy: 0.9720
Epoch 3/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.1105 - prediction1_loss: 0.1117 - prediction2_loss: 0.1092 - prediction1_accuracy: 0.9660 - prediction2_accuracy: 0.9669 - val_loss: 0.0806 - val_prediction1_loss: 0.0811 - val_prediction2_loss: 0.0801 - val_prediction1_accur

In [31]:
# cf. slice
import numpy as np

data = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90,])
data2=np.array(data).reshape(3,3) #3×3の配列を作成
data2

array([[10, 20, 30],
       [40, 50, 60],
       [70, 80, 90]])

In [20]:
data2[:2,] #0～1行目、すべての列

array([[10, 20, 30],
       [40, 50, 60]])

In [26]:
data2[:,:1] #　全ての行、1列目以降

array([[10],
       [40],
       [70]])

In [21]:
data2[:,1:] #すべての行、1列目以降

array([[20, 30],
       [50, 60],
       [80, 90]])

In [24]:
data2[1:,1:] #1行目以降、1列目以降

array([[50, 60],
       [80, 90]])